In [ ]:
# train_intent_classifier.py
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch

class IntentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        encoding = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=32, return_tensors="pt")
        item = {key: val.squeeze() for key, val in encoding.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.texts)

# Sample training
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
texts = ["move forward", "turn left", "stop", "turn right"]
labels = [0, 1, 2, 3]
dataset = IntentDataset(texts, labels, tokenizer)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(5):
    for batch in dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

torch.save(model.state_dict(), "models/intent_model.pt")


The above code is used for training the model

Intent Recognition using BERT (Bidirectional Encoder Representations from Transformers).
Intent recognition is the task of classifying what the user intends to do based on their input — e.g., in a chatbot:

"What's the weather like?" → get_weather

"Play some music" → play_music

"Book a flight" → book_flight


The below code represent the trained model which you had saved after training



In [ ]:
# intent_classifier.py
from transformers import BertTokenizer, BertForSequenceClassification
import torch

intent_labels = ["MOVE_FORWARD", "TURN_LEFT", "STOP", "TURN_RIGHT"]

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
model.load_state_dict(torch.load("models/intent_model.pt"))
model.eval()

def predict_intent(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=32)
    with torch.no_grad():
        outputs = model(**inputs)
        predicted = torch.argmax(outputs.logits, dim=1)
    return intent_labels[predicted.item()]